In [254]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [255]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [256]:
#load data
df = pd.read_csv("loans.csv",index_col=0 )
df.head()
df.shape

/var/folders/bz/dmkkflj524g_9c1psjd1y3rw0000gn/T/ipykernel_4738/131484996.py:2: DtypeWarning: Columns (20,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("loans.csv",index_col=0 )


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
1,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(887379, 74)

In [257]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 887379 entries, 1 to 887379
Data columns (total 74 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           887379 non-null  int64  
 1   member_id                    887379 non-null  int64  
 2   loan_amnt                    887379 non-null  int64  
 3   funded_amnt                  887379 non-null  int64  
 4   funded_amnt_inv              887379 non-null  float64
 5   term                         887379 non-null  object 
 6   int_rate                     887379 non-null  float64
 7   installment                  887379 non-null  float64
 8   grade                        887379 non-null  object 
 9   sub_grade                    887379 non-null  object 
 10  emp_title                    835917 non-null  object 
 11  emp_length                   842554 non-null  object 
 12  home_ownership               887379 non-null  object 
 13  annu

In [258]:
df = df[df.loan_status != 'Current'] # Drops loans with loan status 'Current'
df.loan_status.value_counts()

loan_status
Fully Paid                                             207723
Charged Off                                             45248
Late (31-120 days)                                      11591
Issued                                                   8460
In Grace Period                                          6253
Late (16-30 days)                                        2357
Does not meet the credit policy. Status:Fully Paid       1988
Default                                                  1219
Does not meet the credit policy. Status:Charged Off       761
Name: count, dtype: int64

# drop irrelevant columns 

In [259]:
df.columns 
# Dropping more columns that likely will not be strong predictors

df = df.drop(['emp_title','zip_code','title','url','id','member_id','next_pymnt_d','sub_grade',
             'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
              'funded_amnt', 'funded_amnt_inv', 'pymnt_plan', 'addr_state', 'initial_list_status', 
                'out_prncp', 'out_prncp_inv', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
                'last_credit_pull_d', 'policy_code','issue_d','verification_status','purpose','earliest_cr_line',
              
             ],axis=1)


Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose',
       'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'annual_inc_joint', 'dti_joint', 'verification_status_joint',
    

In [260]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 285600 entries, 1 to 887372
Data columns (total 45 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   loan_amnt                    285600 non-null  int64  
 1   term                         285600 non-null  object 
 2   int_rate                     285600 non-null  float64
 3   installment                  285600 non-null  float64
 4   grade                        285600 non-null  object 
 5   emp_length                   273868 non-null  object 
 6   home_ownership               285600 non-null  object 
 7   annual_inc                   285596 non-null  float64
 8   loan_status                  285600 non-null  object 
 9   desc                         92659 non-null   object 
 10  dti                          285600 non-null  float64
 11  delinq_2yrs                  285571 non-null  float64
 12  inq_last_6mths               285571 non-null  float64
 13  mths

# convert loan_status to [1,0] varible

In [261]:
#Set 'Fully Paid' and 'Does not meet the credit policy. Status:Fully Paid' to 1 and the rest to 0
df['loan_status'] = df['loan_status'].replace({'Fully Paid': 1,'Charged Off': 0, 'Late (31-120 days)': 0,'Late (16-30 days)':0,'Default':0,'Does not meet the credit policy. Status:Charged Off':0,'Does not meet the credit policy. Status:Fully Paid':1})

# Drop 'Issued' and ' In Grace Period' due to lack of information about whether loans are 'good' or 'bad'
df = df[~df['loan_status'].isin(['Issued', 'In Grace Period'])]
df.loan_status.value_counts()

loan_status
1    209711
0     61176
Name: count, dtype: int64

# Missing value

In [262]:
from sklearn.impute import SimpleImputer
df2 = df.copy()

# Drop columns with more than 50% missing values
threshold = 0.5
df2 = df2.dropna(axis=1, thresh=int((1-threshold) * len(df2)))

# Separate numerical and categorical columns
numerical_columns = df2.select_dtypes(include=['number']).columns
categorical_columns = df2.select_dtypes(include=['object']).columns

# Impute missing values for numerical columns with mean
num_imputer = SimpleImputer(strategy='mean')
df2[numerical_columns] = num_imputer.fit_transform(df2[numerical_columns]).copy()

# Impute missing values for categorical columns with most frequent value
cat_imputer = SimpleImputer(strategy='most_frequent')
df2[categorical_columns] = cat_imputer.fit_transform(df2[categorical_columns]).copy()

# Check final shape
print("df2 shape after partial drop + imputation:", df2.shape)

df2 shape after partial drop + imputation: (270887, 24)


In [263]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 270887 entries, 1 to 887372
Data columns (total 24 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   loan_amnt                   270887 non-null  float64
 1   term                        270887 non-null  object 
 2   int_rate                    270887 non-null  float64
 3   installment                 270887 non-null  float64
 4   grade                       270887 non-null  object 
 5   emp_length                  270887 non-null  object 
 6   home_ownership              270887 non-null  object 
 7   annual_inc                  270887 non-null  float64
 8   loan_status                 270887 non-null  object 
 9   dti                         270887 non-null  float64
 10  delinq_2yrs                 270887 non-null  float64
 11  inq_last_6mths              270887 non-null  float64
 12  open_acc                    270887 non-null  float64
 13  pub_rec            

# Separating categorical and continuous variables

In [264]:
categorical_vars = df2.select_dtypes(include=['object']).columns.tolist()
continuous_vars = df2.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_vars

['term',
 'grade',
 'emp_length',
 'home_ownership',
 'loan_status',
 'application_type']

# OneHotEncoder

In [265]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder


# exclued loan_status variable
categorical_var = ['term',
 'grade',
 'emp_length',
 'home_ownership',
 'application_type']

# Initialize OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Apply OneHotEncoding
encoded_cats = encoder.fit_transform(df2[categorical_var])

# Convert to DataFrame
encoded_df2 = pd.DataFrame(encoded_cats, columns=encoder.get_feature_names_out(categorical_var))

# Drop original categorical columns and concatenate encoded ones
df2 = df2.drop(columns=categorical_var).reset_index(drop=True)
df2 = pd.concat([df2, encoded_df2], axis=1)

print("df2 shape after one-hot encoding:", df2.shape)
df2

df2 shape after one-hot encoding: (270887, 47)


,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,emp_length_9 years,emp_length_< 1 year,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,application_type_INDIVIDUAL,application_type_JOINT
0,5000.0,10.65,162.87,24000.0,1,27.65,0.0,1.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,2500.0,15.27,59.83,30000.0,0,1.00,0.0,5.0,3.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,2400.0,15.96,84.33,12252.0,1,8.72,0.0,2.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,10000.0,13.49,339.31,49200.0,1,20.00,0.0,1.0,10.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,5000.0,7.90,156.46,36000.0,1,11.20,0.0,3.0,9.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270882,4200.0,15.99,147.64,48000.0,0,36.93,0.0,0.0,13.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
270883,10775.0,6.03,327.95,54000.0,1,13.22,1.0,0.0,9.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
270884,6225.0,16.49,220.37,27000.0,1,18.58,0.0,1.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
270885,4000.0,8.67,126.59,50000.0,1,12.63,0.0,0.0,11.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


# StandardScaler

In [266]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
common_numerical2 = df2.columns.intersection(continuous_vars)
df2[common_numerical2] = scaler.fit_transform(df2[continuous_vars])

# Imbalance Data 

In [267]:
df2.loan_status.value_counts()

loan_status
1    209711
0     61176
Name: count, dtype: int64

In [268]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270887 entries, 0 to 270886
Data columns (total 47 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   loan_amnt                    270887 non-null  float64
 1   int_rate                     270887 non-null  float64
 2   installment                  270887 non-null  float64
 3   annual_inc                   270887 non-null  float64
 4   loan_status                  270887 non-null  object 
 5   dti                          270887 non-null  float64
 6   delinq_2yrs                  270887 non-null  float64
 7   inq_last_6mths               270887 non-null  float64
 8   open_acc                     270887 non-null  float64
 9   pub_rec                      270887 non-null  float64
 10  revol_bal                    270887 non-null  float64
 11  revol_util                   270887 non-null  float64
 12  total_acc                    270887 non-null  float64
 13 

In [269]:
df2['loan_status'] = df2['loan_status'].astype(int)

In [270]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Define target variable
target = 'loan_status'

X2 = df2.drop(columns=[target])
y2 = df2[target]

#Apply SMOTE separately to df1 and df2
smote = SMOTE(sampling_strategy='auto', random_state=42)

X2_resampled, y2_resampled = smote.fit_resample(X2, y2)

# Convert back to DataFrame

df2_balanced = pd.DataFrame(X2_resampled, columns=X2.columns)
df2_balanced[target] = y2_resampled

In [271]:
df2_balanced.loan_status.value_counts()

loan_status
1    209711
0    209711
Name: count, dtype: int64

# Data Spliting

In [272]:
target = 'loan_status'
X2 = df2.drop(columns=[target])
y2 = df2[target]
X_train2,X_test2,y_train2,y_test2 = train_test_split(X2,y2,test_size=0.3,random_state=42,stratify=y2)

# Model Building 

In [273]:
# Logistic Regression
import sklearn as sk
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

# Initialize Logistic Regression model correctly
clf = LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000,random_state=42)

# Train the model
clf.fit(X2, y2)

# Model score (training data)
train_score = clf.score(X2, y2)

# Predictions on test set
y_pred = clf.predict(X_test2)

# Model evaluation
accuracy = accuracy_score(y_test2, y_pred)
report = classification_report(y_test2, y_pred)

# Display results
train_score
accuracy


LogisticRegression(max_iter=1000, penalty=None, random_state=42)

0.9763997534027103

0.9753282390146062

In [274]:
# decision Tree Model 
